[<img src='https://github.com/jeshraghian/snntorch/blob/master/docs/_static/img/snntorch_alpha_w.png?raw=true' width="400">](https://github.com/jeshraghian/snntorch/)


# snnTorch - Training Spiking Convolutional Neural Networks with snnTorch
## Tutorial 5
### By Jason K. Eshraghian (www.jasoneshraghian.com)

<a href="https://colab.research.google.com/github/jeshraghian/snntorch/blob/master/examples/tutorial_6_CNN.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


[<img src='https://github.com/jeshraghian/snntorch/blob/master/docs/_static/img/GitHub-Mark-Light-120px-plus.png?raw=true' width="28">](https://github.com/jeshraghian/snntorch/) [<img src='https://github.com/jeshraghian/snntorch/blob/master/docs/_static/img/GitHub_Logo_White.png?raw=true' width="80">](https://github.com/jeshraghian/snntorch/)

The snnTorch tutorial series is based on the following paper. If you find these resources or code useful in your work, please consider citing the following source:

> <cite> [Jason K. Eshraghian, Max Ward, Emre Neftci, Xinxin Wang, Gregor Lenz, Girish Dwivedi, Mohammed Bennamoun, Doo Seok Jeong, and Wei D. Lu. "Training Spiking Neural Networks Using Lessons From Deep Learning". arXiv preprint arXiv:2109.12894, September 2021.](https://arxiv.org/abs/2109.12894) </cite>



# Introduction
In this tutorial, you will:
* Learn how spiking neurons are implemented as a recurrent network
* Understand backpropagation through time, and the associated challenges in SNNs such as the non-differentiability of spikes
* Train a fully-connected network on the static MNIST dataset

<!-- * Implement various backprop strategies:
  * Backpropagation Through Time
  * Truncated-Backpropagation Through Time
  * Real-Time Recurrent Learning -->

>Part of this tutorial was inspired by Friedemann Zenke's extensive work on SNNs. Check out his repo on surrogate gradients [here](https://github.com/fzenke/spytorch), and a favourite paper of mine: E. O. Neftci, H. Mostafa, F. Zenke, [Surrogate Gradient Learning in Spiking Neural Networks: Bringing the Power of Gradient-based optimization to spiking neural networks.](https://ieeexplore.ieee.org/document/8891809) IEEE Signal Processing Magazine 36, 51–63.

At the end of the tutorial, a basic supervised learning algorithm will be implemented. We will use the original static MNIST dataset and train a multi-layer fully-connected spiking neural network using gradient descent to perform image classification. 

If running in Google Colab:
* You may connect to GPU by checking `Runtime` > `Change runtime type` > `Hardware accelerator: GPU`
* Next, install the latest PyPi distribution of snnTorch by clicking into the following cell and pressing `Shift+Enter`.

In [1]:
# imports
import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
import numpy as np
import itertools

/Volumes/export/isn/keli/.cache/pypoetry/virtualenvs/l2s-qKZNf-9r-py3.9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Volumes/export/isn/keli/.cache/pypoetry/virtualenvs/l2s-qKZNf-9r-py3.9/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /Volumes/export/isn/keli/.cache/pypoetry/virtualenvs/l2s-qKZNf-9r-py3.9/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIlEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  warn(f"Failed to load image Python extension: {e}")


# 1. A Recurrent Representation of SNNs

In Tutorial 3, we derived a recursive representation of a leaky integrate-and-fire (LIF) neuron: 

$$U[t+1] = \underbrace{\beta U[t]}_\text{decay} + \underbrace{WX[t+1]}_\text{input} - \underbrace{R[t]}_\text{reset} \tag{1}$$

where the input synaptic current is interpreted as $I_{\rm in}[t] = WX[t]$, and $X[t]$ may be some arbitrary input of spikes, a step/time-varying voltage, or unweighted step/time-varying current. Spiking is represented with the following equation, where if the membrane potential exceeds the threshold, a spike is emitted:

$$S[t] = \begin{cases} 1, &\text{if}~U[t] > U_{\rm thr} \\
0, &\text{otherwise}\end{cases} \tag{2}$$

This formulation of a spiking neuron in a discrete, recursive form is almost perfectly poised to take advantage of the developments in training recurrent neural networks (RNNs) and sequence-based models. This is illustrated using an *implicit* recurrent connection for the decay of the membrane potential, and is distinguished from *explicit* recurrence where the output spike $S_{\rm out}$ is fed back to the input. In the figure below, the connection weighted by $-U_{\rm thr}$ represents the reset mechanism $R[t]$.


<center>
<img src='https://github.com/jeshraghian/snntorch/blob/master/docs/_static/img/examples/tutorial5/unrolled_2.png?raw=true' width="800">
</center>

The benefit of an unrolled graph is that it provides an explicit description of how computations are performed. The process of unfolding illustrates the flow of information forward in time (from left to right) to compute outputs and losses, and backward in time to compute gradients. The more time steps that are simulated, the deeper the graph becomes. 

Conventional RNNs treat $\beta$ as a learnable parameter. This is also possible for SNNs, though by default, they are treated as hyperparameters. This replaces the vanishing and exploding gradient problems with a hyperparameter search. A future tutorial will describe how to make $\beta$ a learnable parameter.

# 2. The Non-Differentiability of Spikes
## 2.1 Training Using the Backprop Algorithm

An alternative way to represent the relationship between $S$ and $U$ in $(2)$ is:

$$S[t] = \Theta(U[t] - U_{\rm thr}) \tag{3}$$ 

where $\Theta(\cdot)$ is the Heaviside step function:

<center>
<img src='https://github.com/jeshraghian/snntorch/blob/master/docs/_static/img/examples/tutorial3/3_2_spike_descrip.png?raw=true' width="600">
</center>

Training a network in this form poses some serious challenges. Consider a single, isolated time step of the computational graph from the previous figure titled *"Recurrent representation of spiking neurons"*, as shown in the *forward pass* below:

<center>
<img src='https://github.com/jeshraghian/snntorch/blob/master/docs/_static/img/examples/tutorial5/non-diff.png?raw=true' width="400">
</center>

The goal is to train the network using the gradient of the loss with respect to the weights, such that the weights are updated to minimize the loss. The backpropagation algorithm achieves this using the chain rule:

$$\frac{\partial \mathcal{L}}{\partial W} = 
\frac{\partial \mathcal{L}}{\partial S}
\underbrace{\frac{\partial S}{\partial U}}_{\{0, \infty\}}
\frac{\partial U}{\partial I}\
\frac{\partial I}{\partial W}\ \tag{4}$$

From $(1)$, $\partial I/\partial W=X$, and $\partial U/\partial I=1$. While we have not yet defined a loss function, we can assume $\partial \mathcal{L}/\partial S$ has an analytical solution, in a similar form to the cross-entropy or mean-square error loss (more on that shortly). 

However, the term that we are going to grapple with is $\partial S/\partial U$. The derivative of the Heaviside step function from $(3)$ is the Dirac Delta function, which evaluates to 0 everywhere, except at the threshold $U_{\rm thr} = \theta$, where it tends to infinity. This means the gradient will almost always be nulled to zero (or saturated if $U$ sits precisely at the threshold), and no learning can take place. This is known as the **dead neuron problem**. 

## 2.2 Overcoming the Dead Neuron Problem

The most common way to address the dead neuron problem is to keep the Heaviside function as it is during the forward pass, but swap the derivative term $\partial S/\partial U$ for something that does not kill the learning process during the backward pass, which will be denoted $\partial \tilde{S}/\partial U$. This might sound odd, but it turns out that neural networks are quite robust to such approximations. This is commonly known as the *surrogate gradient* approach.

A variety of options exist to using surrogate gradients, and we will dive into more detail on these methods in [Tutorial 6](https://snntorch.readthedocs.io/en/latest/tutorials/index.html). For now, a simple approximation is applied where $\partial \tilde{S}/\partial U$ is set to $S$ itself.

If $S$ does not spike, then the spike-gradient term is $0$. If $S$ spikes, then the gradient term is $1$. This simply looks like the gradient of a ReLU function shifted to the threshold. This method is known as the *Spike-Operator* approach and is described in more detail in the following paper:

> <cite> Jason K. Eshraghian, Max Ward, Xinxin Wang, Gregor Lenz, Girish Dwivedi, Mohammed Bennamoun, Doo Seok Jeong, and Wei D. Lu. "Training Spiking Neural Networks Using Lessons From Deep Learning". arXiv, 2021. </cite>

Inutitively, *Spike Operator* splits the gradient calculation into two chunks: one where the neuron is spiking, and one where it is silent:
* **Silent:** If the neuron is silent, then the spike response can be obtained by scaling the membrane by 0: $S = U \times 0 \implies \partial \tilde{S}/\partial U = 0$. 
* **Spiking:** If the neuron is spiking, then assume $U \approx U_{\rm thr}$, normalize $U_{\rm thr}=1$, and the spike response can be obtained by scaling the membrane by 1: $S = U \times 1 \implies \partial \tilde{S}/\partial U = 1$, where the tilde above $\tilde{S}$ implies an approximation.

This is summarized as follows:

$$\frac{\partial \tilde{S}}{\partial U} \leftarrow S = \begin{cases} 1, &\text{if}~U> U_{\rm thr} \\
0, &\text{otherwise}\end{cases} $$

where the left arrow denotes substitution. 

The same neuron model described in  $(1)-(2)$ (a.k.a., `snn.Leaky` neuron from Tutorial 3) is implemented in PyTorch below. Don't worry if you don't understand this. This will be condensed into one line of code using snnTorch in a moment:

In [2]:
# Leaky neuron model, overriding the backward pass with a custom function
class LeakySurrogate(nn.Module):
  def __init__(self, beta, threshold=1.0):
      super(LeakySurrogate, self).__init__()

      # initialize decay rate beta and threshold
      self.beta = beta
      self.threshold = threshold
      self.spike_op = self.SpikeOperator.apply
  
  # the forward function is called each time we call Leaky
  def forward(self, input_, mem):
    spk = self.spike_op((mem-self.threshold))  # call the Heaviside function
    reset = (spk * self.threshold).detach() # removes spike_op gradient from reset
    mem = self.beta * mem + input_ - reset # Eq (1)
    return spk, mem

  # Forward pass: Heaviside function
  # Backward pass: Override Dirac Delta with the Spike itself
  @staticmethod
  class SpikeOperator(torch.autograd.Function):
      @staticmethod
      def forward(ctx, mem):
          spk = (mem > 0).float() # Heaviside on the forward pass: Eq(2)
          ctx.save_for_backward(spk)  # store the spike for use in the backward pass
          return spk

      @staticmethod
      def backward(ctx, grad_output):
          (spk,) = ctx.saved_tensors  # retrieve the spike 
          grad = grad_output * spk # scale the gradient by the spike: 1/0
          return grad

Note that the reset mechanism is detached from the computational graph, as the surrogate gradient should only be applied to $\partial S/\partial U$, and not $\partial R/\partial U$.

The above neuron is instantiated using:

In [3]:
lif1 = LeakySurrogate(beta=0.9)

This neuron can be simulated using a for-loop, just as in previous tutorials, while PyTorch's automatic differentation (autodiff) mechanism keeps track of the gradient in the background.

Alternatively, the same thing can be accomplished by calling the `snn.Leaky` neuron. 
In fact, every time you call any neuron model from snnTorch, the *Spike Operator* surrogate gradient is applied to it by default:

In [4]:
#beta is set to zero
lif1 = snn.Leaky(beta=0.9)

If you would like to explore how this neuron behaves, then refer to [Tutorial 3](https://snntorch.readthedocs.io/en/latest/tutorials/index.html).

#3. Backprop Through Time
Equation $(4)$ only calculates the gradient for one single time step (referred to as the *immediate influence* in the figure below), but the backpropagation through time (BPTT) algorithm calculates the gradient from the loss to *all* descendants and sums them together. 

The weight $W$ is applied at every time step, and so imagine a loss is also calculated at every time step. The influence of the weight on present and historical losses must be summed together to define the global gradient:

$$\frac{\partial \mathcal{L}}{\partial W}=\sum_t \frac{\partial\mathcal{L}[t]}{\partial W} = 
\sum_t \sum_{s\leq t} \frac{\partial\mathcal{L}[t]}{\partial W[s]}\frac{\partial W[s]}{\partial W} \tag{5} $$

The point of $(5)$ is to ensure causality: by constraining $s\leq t$, we only account for the contribution of immediate and prior influences of $W$ on the loss. A recurrent system constrains the weight to be shared across all steps: $W[0]=W[1] =~... ~ = W$. Therefore, a change in $W[s]$ will have the same effect on all $W$, which implies that $\partial W[s]/\partial W=1$:

$$\frac{\partial \mathcal{L}}{\partial W}=
\sum_t \sum_{s\leq t} \frac{\partial\mathcal{L}[t]}{\partial W[s]} \tag{6} $$

As an example, isolate the prior influence due to $s = t-1$ *only*; this means the backward pass must track back in time by one step. The influence of $W[t-1]$ on the loss can be written as:

$$\frac{\partial \mathcal{L}[t]}{\partial W[t-1]} = 
\frac{\partial \mathcal{L}[t]}{\partial S[t]}
\underbrace{\frac{\partial \tilde{S}[t]}{\partial U[t]}}_{S[t]}
\underbrace{\frac{\partial U[t]}{\partial U[t-1]}}_\beta
\underbrace{\frac{\partial U[t-1]}{\partial I[t-1]}}_1
\underbrace{\frac{\partial I[t-1]}{\partial W[t-1]}}_{X[t-1]} \tag{7}$$

We have already dealt with all of these terms from $(4)$, except for $\partial U[t]/\partial U[t-1]$. From $(1)$, this temporal derivative term simply evaluates to $\beta$. So if we really wanted to, we now know enough to painstakingly calculate the derivative of every weight at every time step by hand, and it'd look something like this for a single neuron:

<center>
<img src='https://github.com/jeshraghian/snntorch/blob/master/docs/_static/img/examples/tutorial5/bptt.png?raw=true' width="600">
</center>

But thankfully, PyTorch's autodiff takes care of that in the background for us.  

Grip turns a necessity into a fashion statement. And it still works like the high quality Fleck whips that you love! If you are looking for an English riding whip with the security of the Fleck Grip Handle but you also love a bit of sparkle in your life, you're sure to love the Fleck Glitter Riding whip.

Available in pink, lilac and blue, this stylish riding crop measures 45 inches or 60 cm. These Fleck whips are grouped as "Assorted" - if you have a color preference, please note that in the Comments field when you check out. Just # 4. Setting up the Loss / Output Decoding
In a conventional, non-spiking neural network, a supervised, multi-class classification problem takes the neuron with the highest activation and treats that as the predicted class. 

In a spiking neural net, there are several options to interpreting the output spikes. The most common approaches are:
* **Rate coding:** Take the neuron with the highest firing rate (or spike count) as the predicted class
* **Latency coding:** Take the neuron that fires *first* as the predicted class

This might feel familiar to [Tutorial 1 on neural encoding](https://snntorch.readthedocs.io/en/latest/tutorials/index.html). The difference is that, here, we are interpreting (decoding) the output spikes, rather than encoding/converting raw input data into spikes.

Let's focus on a rate code. When input data is passed to the network, we want the correct neuron class to emit the most spikes over the course of the simulation run. This then corresponds to the highest average firing frequency. One way to achieve this is to increase the membrane potential of the correct class to $U>U_{\rm thr}$, and that of incorrect classes to $U<U_{\rm thr}$. Applying the target to $U$ serves as a proxy for modulating spiking behavior from $S$.

This can be implemented by taking the softmax of the membrane potential for output neurons, where $C$ is the number of output classes:

$$p_i[t] = \frac{e^{U_i[t]}}{\sum_{i=0}^{C}e^{U_i[t]}} \tag{8}$$

The cross-entropy between $p_i$ and the target $y_i \in \{0,1\}^C$, which is a one-hot target vector, is obtained using:

$$\mathcal{L}_{CE}[t] = \sum_{i=0}^Cy_i{\rm log}(p_i[t]) \tag{9}$$

The practical effect is that the membrane potential of the correct class is encouraged to increase while those of incorrect classes are reduced. In effect, this means the correct class is encouraged to fire at all time steps, while incorrect classes are suppressed at all steps. This may not be the most efficient implementation of an SNN, but it is among the simplest.

This target is applied at every time step of the simulation, thus also generating a loss at every step. These losses are then summed together at the end of the simulation:

$$\mathcal{L}_{CE} = \sum_t\mathcal{L}_{CE}[t] \tag{10}$$

This is just one of many possible ways to apply a loss function to a spiking neural network. A variety of approaches are available to use in snnTorch (in the module `snn.functional`), and will be the subject of a future tutorial.

With all of the background theory having been taken care of, let’s finally dive into
training a fully-connected spiking neural net.

# 5. Setting up the Static MNIST Dataset

In [2]:
# dataloader arguments
batch_size = 128
data_path='~/justinData/mnistData'

dtype = torch.float
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [3]:
device

device(type='cuda')

In [4]:
# Define a transform
transform = transforms.Compose([
            transforms.Resize((28, 28)),
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize((0,), (1,))])

mnist_train = datasets.MNIST(data_path, train=True, download=True, transform=transform)
mnist_test = datasets.MNIST(data_path, train=False, download=True, transform=transform)

If the above code blocks throws an error, e.g. the MNIST servers are down, then uncomment the following code instead.

In [5]:
# # temporary dataloader if MNIST service is unavailable
# !wget www.di.ens.fr/~lelarge/MNIST.tar.gz
# !tar -zxvf MNIST.tar.gz

# mnist_train = datasets.MNIST(root = './', train=True, download=True, transform=transform)
# mnist_test = datasets.MNIST(root = './', train=False, download=True, transform=transform)

In [6]:
# Create DataLoaders
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=True, drop_last=True)

# 6. Define the Network

In [7]:
# Network Architecture
num_inputs = 28*28
num_hidden = 1000
num_outputs = 10

# Temporal Dynamics
num_steps = 25
#beta is set to 0.0
beta = 1.0

In [8]:
# Define Network
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        # Initialize layers
        self.fc1 = nn.Linear(num_inputs, num_hidden)
        self.lif1 = snn.Leaky(beta=beta)
        
        self.fc2 = nn.Linear(num_hidden, num_outputs)
        self.lif2 = snn.Leaky(beta=beta)

    def forward(self, x):

        # Initialize hidden states at t=0
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        
        # Record the final layer
        spk2_rec = []
        mem2_rec = []
        
        data = spikegen.rate(x,num_steps)

        for q in data:

            cur1 = self.fc1(q)
            spk1, mem1 = self.lif1(cur1, mem1)
            cur2 = self.fc2(spk1)
            spk2, mem2 = self.lif2(cur2, mem2)
            spk2_rec.append(spk2)
            mem2_rec.append(mem2)

        return torch.stack(spk2_rec, dim=0), torch.stack(mem2_rec, dim=0)
        
# Load the network onto CUDA if available
net = Net().to(device)

The code in the `forward()` function will only be called once the input argument `x` is explicitly passed into `net`.

* `fc1` applies a linear transformation to all input pixels from the MNIST dataset;
* `lif1` integrates the weighted input over time, emitting a spike if the threshold condition is met;
* `fc2` applies a linear transformation to the output spikes of `lif1`;
* `lif2` is another spiking neuron layer, integrating the weighted spikes over time.

# 7. Training the SNN

## 7.1 Accuracy Metric
Below is a function that takes a batch of data, counts up all the spikes from each neuron (i.e., a rate code over the simulation time), and compares the index of the highest count with the actual target. If they match, then the network correctly predicted the target.

In [9]:
# pass data into the network, sum the spikes over time
# and compare the neuron with the highest number of spikes
# with the target

def print_batch_accuracy(data, targets, train=False):
    output, _ = net(data.view(batch_size, -1))
    _, idx = output.sum(dim=0).max(1)
    acc = np.mean((targets == idx).detach().cpu().numpy())

    if train:
        print(f"Train set accuracy for a single minibatch: {acc*100:.2f}%")
    else:
        print(f"Test set accuracy for a single minibatch: {acc*100:.2f}%")

def train_printer():
    print(f"Epoch {epoch}, Iteration {iter_counter}")
    print(f"Train Set Loss: {loss_hist[counter]:.2f}")
    print(f"Test Set Loss: {test_loss_hist[counter]:.2f}")
    print_batch_accuracy(data, targets, train=True)
    print_batch_accuracy(test_data, test_targets, train=False)
    print("\n")

## 7.2 Loss Definition
The `nn.CrossEntropyLoss` function in PyTorch automatically handles taking the softmax of the output layer as well as generating a loss at the output. 

In [10]:
loss = nn.CrossEntropyLoss()

## 7.3 Optimizer
Adam is a robust optimizer that performs well on recurrent networks, so let's use that with a learning rate of $5\times10^{-4}$. 

In [11]:
optimizer = torch.optim.Adam(net.parameters(), lr=5e-4, betas=(0.9, 0.999))

## 7.4 One Iteration of Training
Take the first batch of data and load it onto CUDA if available.

In [21]:
data, targets = next(iter(train_loader))
data = data.to(device)
targets = targets.to(device)

In [22]:
data.view(batch_size, -1).shape

torch.Size([128, 784])

Flatten the input data to a vector of size $784$ and pass it into the network.

In [23]:
spk_rec, mem_rec = net(data.view(batch_size, -1))
print(mem_rec.size())

torch.Size([25, 128, 10])


The recording of the membrane potential is taken across:
* 25 time steps
* 128 samples of data
* 10 output neurons

We wish to calculate the loss at every time step, and sum these up together, as per Equation $(10)$:

In [24]:
# initialize the total loss value
loss_val = torch.zeros((1), dtype=dtype, device=device)

# sum loss at every step
for step in range(num_steps):
  loss_val += loss(mem_rec[step], targets)

print(f"Training loss: {loss_val.item():.3f}")

Training loss: 64962.070


The loss is quite large, because it is summed over 25 time steps. The accuracy is also bad (it should be roughly around 10%) as the network is untrained:

In [25]:
print_batch_accuracy(data, targets, train=True)

Train set accuracy for a single minibatch: 96.88%


A single weight update is applied to the network as follows:

In [26]:
# clear previously stored gradients
optimizer.zero_grad()

# calculate the gradients
loss_val.backward()

# weight update
optimizer.step()

Now, re-run the loss calculation and accuracy after a single iteration:

In [27]:
# calculate new network outputs using the same data
spk_rec, mem_rec = net(data.view(batch_size, -1))

# initialize the total loss value
loss_val = torch.zeros((1), dtype=dtype, device=device)

# sum loss at every step
for step in range(num_steps):
  loss_val += loss(mem_rec[step], targets)

print(f"Training loss: {loss_val.item():.3f}")
print_batch_accuracy(data, targets, train=True)

Training loss: 55052.602
Train set accuracy for a single minibatch: 96.09%


After only one iteration, the loss should have decreased and accuracy should have increased. Note how membrane potential is used to calculate the cross entropy
loss, and spike count is used for the measure of accuracy. It is also possible to use the spike count in the loss ([see Tutorial 6](https://snntorch.readthedocs.io/en/latest/tutorials/index.html))

## 7.5 Training Loop

Let's combine everything into a training loop. We will train for one epoch (though feel free to increase `num_epochs`), exposing our network to each sample of data once.

In [23]:
num_epochs = 1
loss_hist = []
test_loss_hist = []
counter = 0

# Outer training loop
for epoch in range(num_epochs):
    iter_counter = 0
    train_batch = iter(train_loader)

    # Minibatch training loop
    for data, targets in train_batch:
        data = data.to(device)
        targets = targets.to(device)

        # forward pass
        net.train()
        spk_rec, mem_rec = net(data.view(batch_size, -1))

        # initialize the loss & sum over time
        loss_val = torch.zeros((1), dtype=dtype, device=device)
        for step in range(num_steps):
            loss_val += loss(mem_rec[step], targets)

        # Gradient calculation + weight update
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()

        # Store loss history for future plotting
        loss_hist.append(loss_val.item())

        # Test set
        with torch.no_grad():
            net.eval()
            test_data, test_targets = next(iter(test_loader))
            test_data = test_data.to(device)
            test_targets = test_targets.to(device)

            # Test set forward pass
            test_spk, test_mem = net(test_data.view(batch_size, -1))

            # Test set loss
            test_loss = torch.zeros((1), dtype=dtype, device=device)
            for step in range(num_steps):
                test_loss += loss(test_mem[step], test_targets)
            test_loss_hist.append(test_loss.item())

            # Print train/test loss/accuracy
            if counter % 50 == 0:
                train_printer()
            counter += 1
            iter_counter +=1

Epoch 0, Iteration 0
Train Set Loss: 54.35
Test Set Loss: 48.83
Train set accuracy for a single minibatch: 57.03%
Test set accuracy for a single minibatch: 46.09%


Epoch 0, Iteration 50
Train Set Loss: 14.71
Test Set Loss: 17.26
Train set accuracy for a single minibatch: 89.84%
Test set accuracy for a single minibatch: 87.50%


Epoch 0, Iteration 100
Train Set Loss: 13.15
Test Set Loss: 14.84
Train set accuracy for a single minibatch: 89.06%
Test set accuracy for a single minibatch: 91.41%


Epoch 0, Iteration 150
Train Set Loss: 9.71
Test Set Loss: 8.20
Train set accuracy for a single minibatch: 93.75%
Test set accuracy for a single minibatch: 95.31%


Epoch 0, Iteration 200
Train Set Loss: 10.49
Test Set Loss: 6.80
Train set accuracy for a single minibatch: 88.28%
Test set accuracy for a single minibatch: 91.41%


Epoch 0, Iteration 250
Train Set Loss: 8.08
Test Set Loss: 8.28
Train set accuracy for a single minibatch: 92.19%
Test set accuracy for a single minibatch: 90.62%


Epoch 

If this was your first time training an SNN, then congratulations!

In [28]:
print(list(net.fc1.parameters())[0].detach().cpu().numpy().shape)
print(list(net.fc2.parameters())[0].detach().cpu().numpy().shape)

(1000, 784)
(10, 1000)


## Quantization of the trained model

In [29]:

layer1 = torch.quantize_per_tensor(
    net.fc1.weight,  # the original model
    torch.tensor(torch.max(net.fc1.weight)/32767).to(device), #scale
    torch.tensor(0).to(device), #zero_point
    dtype=torch.qint8)  # the target dtype for quantized weights

/tmp/ipykernel_131391/3881061420.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(torch.max(net.fc1.weight)/32767).to(device), #scale


In [30]:
def weight_quantization(b):

    def uniform_quant(x, b):
        xdiv = x.mul((2 ** b - 1))
        xhard = xdiv.round().div(2 ** b - 1)
        #print('uniform quant bit: ', b)
        return xhard

    class _pq(torch.autograd.Function):
        @staticmethod
        def forward(ctx, input, alpha):
            input.div_(alpha)                          # weights are first divided by alpha
            input_c = input.clamp(min=-1, max=1)       # then clipped to [-1,1]
            sign = input_c.sign()
            input_abs = input_c.abs()
            input_q = uniform_quant(input_abs, b).mul(sign)
            ctx.save_for_backward(input, input_q)
            input_q = input_q.mul(alpha)               # rescale to the original range
            return input_q

        @staticmethod
        def backward(ctx, grad_output):
            grad_input = grad_output.clone()             # grad for weights will not be clipped
            input, input_q = ctx.saved_tensors
            i = (input.abs()>1.).float()     # >1 means clipped. # output matrix is a form of [True, False, True, ...]
            sign = input.sign()              # output matrix is a form of [+1, -1, -1, +1, ...]
            #grad_alpha = (grad_output*(sign*i + (input_q-input)*(1-i))).sum()
            grad_alpha = (grad_output*(sign*i + (0.0)*(1-i))).sum()
            # above line, if i = True,  and sign = +1, "grad_alpha = grad_output * 1"
            #             if i = False, "grad_alpha = grad_output * (input_q-input)"
            grad_input = grad_input*(1-i)
            return grad_input, grad_alpha

    return _pq().apply

class weight_quantize_fn(nn.Module):
    def __init__(self, w_bit):
        super(weight_quantize_fn, self).__init__()
        self.w_bit = w_bit-1
        self.weight_q = weight_quantization(b=self.w_bit)
        self.register_parameter('wgt_alpha', Parameter(torch.tensor(3.0)))

    def forward(self, weight):
        mean = weight.data.mean()
        std = weight.data.std()
        weight = weight.add(-mean).div(std)      # weights normalization
        weight_q = self.weight_q(weight, self.wgt_alpha)

        return weight_q

### Load saved model

In [12]:
best_model_path = '/Volumes/export/isn/gopa/CRI_proj/L2S_justin/snntorch_mnist/result/mnist_2layer_MLP_quantized/model_best.pth.tar'
checkpoint = torch.load(best_model_path)
model_best = net
model_best.load_state_dict(checkpoint['state_dict'])
model_best.eval()


Net(
  (fc1): Linear(in_features=784, out_features=1000, bias=True)
  (lif1): Leaky()
  (fc2): Linear(in_features=1000, out_features=10, bias=True)
  (lif2): Leaky()
)

# 8. Results
## 8.1 Plot Training/Test Loss

In [13]:
# Plot Loss
fig = plt.figure(facecolor="w", figsize=(10, 5))
plt.plot(loss_hist)
plt.plot(test_loss_hist)
plt.title("Loss Curves")
plt.legend(["Train Loss", "Test Loss"])
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.show()

NameError: name 'loss_hist' is not defined

<Figure size 1000x500 with 0 Axes>

The loss curves are noisy because the losses are tracked at every iteration, rather than averaging across multiple iterations. 

## 8.2 Test Set Accuracy
This function iterates over all minibatches to obtain a measure of accuracy over the full 10,000 samples in the test set.

In [14]:
total = 0
correct = 0

# drop_last switched to False to keep all samples
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=True, drop_last=False)

with torch.no_grad():
  net.eval()
  for data, targets in test_loader:
    data = data.to(device)
    targets = targets.to(device)
    
    # forward pass
    test_spk, _ = net(data.view(data.size(0), -1))

    # calculate total accuracy
    _, predicted = test_spk.sum(dim=0).max(1)
    total += targets.size(0)
    correct += (predicted == targets).sum().item()

print(f"Total correctly classified test set images: {correct}/{total}")
print(f"Test Set Accuracy: {100 * correct / total:.2f}%")


Total correctly classified test set images: 9649/10000
Test Set Accuracy: 96.49%


Voila! That's it for static MNIST. Feel free to tweak the network parameters, hyperparameters, decay rate, using a learning rate scheduler etc. to see if you can improve the network performance. 

### Map trained SNN from torchsnn to CRI 

In [15]:
# extract weights and bias for torchsnn
layers = [list(model_best.fc1.parameters())[0].detach().cpu().numpy(), list(model_best.fc2.parameters())[0].detach().cpu().numpy()]
biases = [model_best.fc1.bias.detach().cpu().numpy(), model_best.fc2.bias.detach().cpu().numpy()]

print(np.min(layers[1]))
print(np.max(layers[1]))


-4160.0
3713.0


In [16]:
# scale = 9*10**4
axonsDict = {}
neuronsDict = {}
outputs = []
bias_axon = {}

axonOffset = 0
currLayerNeuronIdxOffset = 0
nextLayerNeuronIdxOffset = 0
for layerNum, layer in enumerate(layers):
    inFeatures = layer.shape[1]
    outFeatures = layer.shape[0]
    shape = layer.shape
    weight = layer
    bias = biases[layerNum]
    print("Weights shape: ", np.shape(weight))
    if (layerNum == 0):
        print('constructing Axons')
        print("Input layer shape(outfeature, infeature): ", weight.shape)
        for axonIdx, axon in enumerate(weight.T):
            #print(axonIdx)
            axonID = 'a'+str(axonIdx)
            axonEntry = [(str(postSynapticID), int(synapseWeight)) for postSynapticID, synapseWeight in enumerate(axon) ]
            axonsDict[axonID] = axonEntry
        axonOffset += inFeatures
        print("axon offset: ",axonOffset)
        #implmenting bias: for each bias add a axon with corresponding weights with synapse (neuron, bias_val)
        print('Construct bias axons for hidden layers:',bias.shape)
        for neuronIdx, bias_value in enumerate(bias):
            biasAxonID = 'a'+str(neuronIdx + axonOffset)
            biasAxonEntry = [(str(neuronIdx),int(bias_value))]
            axonsDict[biasAxonID] = biasAxonEntry
        
    elif (layerNum == len(layers)-1):
        print('constructing output layer')
        nextLayerNeuronIdxOffset += inFeatures
        print("output layer shape(outfeature, infeature): ", weight.shape)
        for baseNeuronIdx, neuron in enumerate(weight.T):
            neuronID = str(baseNeuronIdx+currLayerNeuronIdxOffset)
            neuronEntry = [(str(basePostSynapticID+nextLayerNeuronIdxOffset), int(synapseWeight)) for basePostSynapticID, synapseWeight in enumerate(neuron) if synapseWeight != 0]
            neuronsDict[neuronID] = neuronEntry
            #print(neuronID)
        currLayerNeuronIdxOffset += inFeatures
        #instantiate the output neurons
        print('instantiate output neurons')
        for baseNeuronIdx in range(outFeatures):
            neuronID = str(baseNeuronIdx+nextLayerNeuronIdxOffset)
            neuronsDict[neuronID] = []
            outputs.append(neuronID)
            #print(neuronID)
        #implmenting bias: for each bias add a axon with corresponding weights with synapse (neuron, bias_val)
        print('Construct bias axons for output neurons',bias.shape)
        axonOffset += inFeatures
        for neuronIdx, bias_value in enumerate(bias):
            biasAxonID = 'a'+str(neuronIdx + axonOffset)
            biasAxonEntry = [(str(neuronIdx+nextLayerNeuronIdxOffset),int(bias_value))]
            axonsDict[biasAxonID] = biasAxonEntry
            
    else:
        print('constructing hidden layer')
        nextLayerNeuronIdxOffset += inFeatures
        for baseNeuronIdx, neuron in enumerate(weight): #SHOULD THIS BE A TRANSPOSE
            neuronID = str(baseNeuronIdx+currLayerNeuronIdxOffset)
            neuronEntry = [(str(basePostSynapticID+nextLayerNeuronIdxOffset), int(synapseWeight)) for basePostSynapticID, synapseWeight in enumerate(neuron) if synapseWeight != 0 ]
            neuronsDict[neuronID] = neuronEntry
            #print(neuronID)
        currLayerNeuronIdxOffset += inFeatures
        axonOffset += inFeatures
        for neuronIdx, bias_value in enumerate(bias):
            biasAxonID = 'a'+str(neuronIdx + axonOffset)
            biasAxonEntry = [(str(neuronIdx+nextLayerNeuronIdxOffset),int(bias_value))]
            axonsDict[biasAxonID] = biasAxonEntry
print("output neurons: ", outputs)
print("number of axons: ", len(axonsDict))

Weights shape:  (1000, 784)
constructing Axons
Input layer shape(outfeature, infeature):  (1000, 784)
axon offset:  784
Construct bias axons for hidden layers: (1000,)
Weights shape:  (10, 1000)
constructing output layer
output layer shape(outfeature, infeature):  (10, 1000)
instantiate output neurons
Construct bias axons for output neurons (10,)
output neurons:  ['1000', '1001', '1002', '1003', '1004', '1005', '1006', '1007', '1008', '1009']
number of axons:  1794


In [17]:
print("Number of axons: ",len(axonsDict))
totalAxonSyn = 0
maxFan = 0
for key in axonsDict.keys():
    totalAxonSyn += len(axonsDict[key])
    if len(axonsDict[key]) > maxFan:
        maxFan = len(axonsDict[key])
print("Total number of connections between axon and neuron: ", totalAxonSyn)
print("Max fan out of axon: ", maxFan)
print('---')
print("Number of neurons: ", len(neuronsDict))
totalSyn = 0
maxFan = 0
for key in neuronsDict.keys():
    totalSyn += len(neuronsDict[key])
    if len(neuronsDict[key]) > maxFan:
        maxFan = len(neuronsDict[key])
print("Total number of connections between hidden and output layers: ", totalSyn)
print("Max fan out of neuron: ", maxFan)
print(neuronsDict['1007'])

Number of axons:  1794
Total number of connections between axon and neuron:  785010
Max fan out of axon:  1000
---
Number of neurons:  1010
Total number of connections between hidden and output layers:  9996
Max fan out of neuron:  10
[]


In [18]:
import time

In [ ]:
from l2s.api import CRI_network
config = {}
config['neuron_type'] = "I&F"
config['global_neuron_params'] = {}
config['global_neuron_params']['v_thr'] = 9*10**4
softwareNetwork = CRI_network(axons=axonsDict,connections=neuronsDict,config=config,target='simpleSim', outputs = outputs)
hardwareNetwork = CRI_network(axons=axonsDict,connections=neuronsDict,config=config,target='CRI', outputs = outputs,simDump = False)

added axons to connectome
added neurons to connectome
added axon synpases
added neuron synapses
generated Connectome
initialized
added axons to connectome
added neurons to connectome
added axon synpases
added neuron synapses
generated Connectome


In [39]:
def input_to_CRI(currentInput):
    num_steps = 10
    currentInput = data.view(data.size(0), -1)
    batch = []
    n = 0
    for element in currentInput:
        timesteps = []
        rateEnc = spikegen.rate(element,num_steps)
        rateEnc = rateEnc.detach().cpu().numpy()
        for element in rateEnc:
            currInput = ['a'+str(idx) for idx,axon in enumerate(element) if axon != 0]
            biasInput = ['a'+str(idx) for idx in range(784,len(axonsDict))]
#             timesteps.append(currInput)
#             timesteps.append(biasInput)
            timesteps.append(currInput+biasInput)
        batch.append(timesteps)
    return batch

In [40]:
def run_CRI(inputList, output_offset):
    predictions = []
    total_time_cri = 0
    #each image
    for currInput in inputList:
        #reset the membrane potential to zero
        softwareNetwork.simpleSim.initialize_sim_vars(len(neuronsDict))
        spikeRate = [0]*10
        #each time step
        for slice in currInput:
            start_time = time.time()
            swSpike = softwareNetwork.step(slice, membranePotential=False)
            end_time = time.time()
            total_time_cri = total_time_cri + end_time-start_time
            for spike in swSpike:
                spikeIdx = int(spike) - output_offset 
                try: 
                    if spikeIdx >= 0: 
                        spikeRate[spikeIdx] += 1 
                except:
                    print("SpikeIdx: ", spikeIdx,"\n SpikeRate:",spikeRate )
        predictions.append(spikeRate.index(max(spikeRate)))
    print(f"Total simulation execution time: {total_time_cri:.5f} s")

    return(predictions)

In [ ]:
def run_CRI_hw(inputList,output_offset):
    predictions = []
    #each image
    total_time_cri = 0
    for currInput in inputList:
        #initiate the softwareNetwork for each image
        cri_simulations.FPGA_Execution.fpga_controller.clear(len(neuronsDict), False, 0)  ##Num_neurons, simDump, coreOverride
        spikeRate = [0]*10
        #each time step
        for slice in currInput:
            start_time = time.time()
            hwSpike = hardwareNetwork.step(slice, membranePotential=False)
#             print("Mem:",mem)
            end_time = time.time()
            total_time_cri = total_time_cri + end_time-start_time
            print(hwSpike)
            for spike in hwSpike:
                print(int(spike))
                spikeIdx = int(spike) - output_offset 
                if spikeIdx >= 0: 
                    spikeRate[spikeIdx] += 1 
        predictions.append(spikeRate.index(max(spikeRate))) 
    print(f"Total execution time CRIFPGA: {total_time_cri:.5f} s")
    return(predictions)

In [41]:
# import pickle

# pickle.dump( axonsDict, open( "axonsDict.p", "wb" ) )
# pickle.dump( neuronsDict, open( "neuronsDict.p", "wb" ) )# pickle.dump( outputs, open( "outputs.p", "wb" ) )

In [42]:
total = 0
correct = 0
cri_correct = 0
# drop_last switched to False to keep all samples
test_loader = DataLoader(mnist_test, batch_size=128, shuffle=True, drop_last=False)

with torch.no_grad():
    net.eval()
    for data, targets in test_loader:
        data = data.to(device)
        targets = targets.to(device)
        input = input_to_CRI(data)
        criPred = torch.tensor(run_CRI(input)).to(device)
        print("CRI Predicted: ",criPred)
        criPred_hw = torch.tensor(run_CRI_hw(input,output_offset)).to(device)
        print("CRI Predicted HW: ",criPred_hw)
        # print(data.shape)
        # forward pass
        test_spk, _ = net(data.view(data.size(0), -1))

        # calculate total accuracy
        _, predicted = test_spk.sum(dim=0).max(1)
        print("Torchsnn Predicted: ",predicted)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
        cri_correct += (criPred == targets).sum().item()
        cri_correct_hw += (criPred_hw == targets).sum().item()
        break #run for one batch

print(f"Total correctly classified test set images for TorchSNN: {correct}/{total}")
print(f"Total correctly classified test set images for CRI: {cri_correct}/{total}")
print(f"Test Set Accuracy for TorchSNN: {100 * correct / total:.2f}%")
print(f"Test Set Accuracy for CRI: {100 * cri_correct / total:.2f}%")

CRI Predicted:  tensor([9, 3, 5, 7, 2, 3, 3, 4, 1, 3, 2, 1, 7, 0, 1, 6, 2, 9, 1, 3, 0, 6, 5, 1,
        5, 9, 1, 1, 8, 1, 2, 9, 5, 4, 2, 1, 0, 7, 2, 9, 8, 8, 6, 1, 3, 0, 0, 9,
        0, 9, 1, 4, 3, 1, 7, 7, 6, 5, 8, 8, 7, 7, 0, 3, 6, 1, 1, 4, 3, 3, 8, 6,
        8, 0, 8, 2, 6, 6, 4, 7, 1, 1, 3, 3, 3, 4, 2, 4, 6, 9, 3, 5, 0, 4, 5, 4,
        2, 1, 8, 5, 0, 4, 0, 1, 7, 0, 1, 8, 0, 5, 9, 0, 1, 2, 0, 8, 1, 1, 9, 8,
        4, 7, 5, 9, 0, 9, 7, 0], device='cuda:0')
Target:  tensor([9, 3, 5, 7, 2, 3, 3, 4, 1, 3, 2, 1, 7, 0, 1, 6, 2, 9, 1, 3, 0, 5, 5, 1,
        5, 9, 1, 1, 8, 1, 2, 9, 5, 4, 2, 1, 3, 7, 2, 9, 8, 8, 6, 1, 3, 0, 0, 9,
        0, 8, 1, 4, 3, 1, 7, 7, 6, 6, 8, 8, 7, 7, 0, 3, 6, 1, 1, 4, 3, 3, 8, 6,
        8, 0, 3, 2, 6, 6, 4, 7, 1, 1, 3, 3, 3, 4, 2, 4, 6, 9, 3, 5, 0, 4, 5, 4,
        2, 1, 8, 5, 0, 4, 0, 1, 7, 4, 1, 8, 4, 5, 9, 0, 1, 2, 3, 8, 1, 1, 9, 8,
        4, 7, 5, 9, 0, 9, 7, 0], device='cuda:0')
Torchsnn Predicted:  tensor([9, 3, 5, 7, 2, 3, 3, 4, 1, 3, 2, 1, 7, 0, 1, 6

In [ ]:
# hardwareNetwork.sim_flush('/home/justinData/jul15Dump.txt')

# Conclusion
Now you know how to construct and train a fully-connected network on a static dataset. The spiking neurons can also be adapted to other layer types, including convolutions and skip connections. Armed with this knowledge, you should now be able to build many different types of SNNs. [In the next tutorial](https://snntorch.readthedocs.io/en/latest/tutorials/index.html), you will learn how to train a spiking convolutional network, and simplify the amount of code required using the `snn.backprop` module.

Also, a special thanks to Bugra Kaytanli for providing valuable feedback on the tutorial.

If you like this project, please consider starring ⭐ the repo on GitHub as it is the easiest and best way to support it.


# Additional Resources

* [Check out the snnTorch GitHub project here.](https://github.com/jeshraghian/snntorch)

Let's try matching a tiny SNNTorch network with the output of CRI

In [ ]:
# dataloader arguments
batch_size = 258
data_path='~/justinData/mnistData'

dtype = torch.float
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
# Define a transform
transform = transforms.Compose([
            transforms.Resize((5,1)), #make the image drastically smaller
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize((0,), (1,))])

mnist_train = datasets.MNIST(data_path, train=True, download=True, transform=transform)
mnist_test = datasets.MNIST(data_path, train=False, download=True, transform=transform)

In [ ]:
# Create DataLoaders
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=True, drop_last=True)

Define the network

In [ ]:
# Network Architecture
num_inputs = 5
num_hidden = 10
num_outputs = 10

# Temporal Dynamics
num_steps = 25
beta = 0
from snntorch import spikeplot as splt

In [ ]:
# Define Network
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        # Initialize layers
        #self.fc1 = nn.Linear(num_inputs, num_hidden, bias=False)
        #self.lif1 = snn.Leaky(beta=beta,reset_mechanism='zero')
        
        #self.fc2 = nn.Linear(num_hidden, num_outputs, bias=False)
        #self.lif2 = snn.Leaky(beta=beta,reset_mechanism='zero')
        self.fc1 = nn.Linear(num_inputs, num_hidden, bias=False)
        self.lif1 = snn.Leaky(beta=beta)
        
        self.fc2 = nn.Linear(num_hidden, num_outputs, bias=False)
        self.lif2 = snn.Leaky(beta=beta)

    def forward(self, x):

        # Initialize hidden states at t=0
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        
        # Record the final layer
        spk2_rec = []
        spk1_rec = []
        mem2_rec = []
        
        #data = spikegen.rate(x,num_steps)

        #for q in data:
        #    cur1 = self.fc1(q)
        #    spk1, mem1 = self.lif1(cur1, mem1)
        #    cur2 = self.fc2(spk1)
        #    spk2, mem2 = self.lif2(cur2, mem2)
        #    spk2_rec.append(spk2)
        #    mem2_rec.append(mem2)

        #return torch.stack(spk2_rec, dim=0), torch.stack(mem2_rec, dim=0)
        # network simulation
        for step in range(num_steps):
            cur1 = self.fc1(x[step]) # post-synaptic current <-- spk_in x weight
            spk1, mem1 = self.lif1(cur1, mem1) # mem[t+1] <--post-syn current + decayed membrane
            cur2 = self.fc2(spk1)
            spk2, mem2 = self.lif2(cur2, mem2)

            mem2_rec.append(mem2)
            spk1_rec.append(spk1)
            spk2_rec.append(spk2)

        # convert lists to tensors
        mem2_rec = torch.stack(mem2_rec)
        spk1_rec = torch.stack(spk1_rec)
        spk2_rec = torch.stack(spk2_rec)
        
        return spk2_rec

       
        
# Load the network onto CUDA if available
net = Net().to(device)


In [ ]:
spk_in = spikegen.rate_conv(torch.rand((200, num_inputs))).unsqueeze(1)

In [ ]:
net(spk_in)

In [ ]:
layers = [list(net.fc1.parameters())[0].detach().cpu().numpy(), list(net.fc2.parameters())[0].detach().cpu().numpy()]
print(layers)


Define lossfunction and optimization

In [ ]:
# pass data into the network, sum the spikes over time
# and compare the neuron with the highest number of spikes
# with the target

def print_batch_accuracy(data, targets, train=False):
    output, _ = net(data.view(batch_size, -1))
    _, idx = output.sum(dim=0).max(1)
    acc = np.mean((targets == idx).detach().cpu().numpy())

    if train:
        print(f"Train set accuracy for a single minibatch: {acc*100:.2f}%")
    else:
        print(f"Test set accuracy for a single minibatch: {acc*100:.2f}%")

def train_printer():
    print(f"Epoch {epoch}, Iteration {iter_counter}")
    print(f"Train Set Loss: {loss_hist[counter]:.2f}")
    print(f"Test Set Loss: {test_loss_hist[counter]:.2f}")
    print_batch_accuracy(data, targets, train=True)
    print_batch_accuracy(test_data, test_targets, train=False)
    print("\n")

In [ ]:
loss = nn.CrossEntropyLoss()

In [ ]:
optimizer = torch.optim.Adam(net.parameters(), lr=5e-4, betas=(0.9, 0.999))

Begin Training

In [ ]:
total = 0
correct = 0

# drop_last switched to False to keep all samples
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=True, drop_last=False)

with torch.no_grad():
  net.eval()
  for data, targets in test_loader:
    print(data.shape)
    data = data.to(device)
    targets = targets.to(device)
    
    # forward pass
    test_spk, _ = net(data.view(data.size(0), -1))
    print(test_spk.shape)
    break
"""
    # calculate total accuracy
    _, predicted = test_spk.sum(dim=0).max(1)
    total += targets.size(0)
    correct += (predicted == targets).sum().item()

print(f"Total correctly classified test set images: {correct}/{total}")
print(f"Test Set Accuracy: {100 * correct / total:.2f}%")
"""